In [1]:
import pandas as pd
from jupyter_demo.utils import query_or_load, set_plot_opts
from jupyter_demo.content.sql_queries import tv_radio_station
from IPython.core.display import display
from jupyter_demo.content.plots import USMap, Distribution, HighLevelTable, MoreDetailTable


In [2]:
set_plot_opts()

### Table of Contents <a name="toc"></a>
* [US Broadcast TV/Radio Stations](#1)
* [City and State Coordinates](#2)
* [Join Coordinates and Stations](#3)
* [High Level Aggregation](#4)
* [More Detail](#5)
* [Media Station Distribution](#6)
* [Geographical Distribution by City](#7")


#### US Broadcast TV/Radio Stations <a name="1"></a>

[back to top](#toc)

In [3]:
print(tv_radio_station.sql_command)


SELECT
    communityState,
    INITCAP(communityCity) as communityCity,
    service,
    status,
    COUNT(DISTINCT stationId) AS number_of_stations
FROM `bigquery-public-data.fcc_political_ads.broadcast_tv_radio_station`
GROUP BY communityState, communityCity, service, status



In [4]:
tv_radio_station_df = query_or_load(module=tv_radio_station, refresh=False)
tv_radio_station_df.head(5)

,communityState,communityCity,service,status,number_of_stations
0,MO,St. Louis,Digital TV,LICENSED,6
1,CA,Mecca,FM Station,LICENSED,3
2,PR,Rio Grande,FM Station,LICENSED,1
3,PR,Quebradillas,FM Station,LICENSED,2
4,ID,Island Park,FM Station,LICENSED,1


#### City and State Coordinates <a name="2"></a>

[back to top](#toc)

In [5]:
geolocation_df = pd.read_csv('../content/data/uscities.csv')
geolocation_df[['city', 'state_id', 'state_name', 'lat','lng']].head(5)

,city,state_id,state_name,lat,lng
0,New York,NY,New York,40.6943,-73.9249
1,Los Angeles,CA,California,34.1139,-118.4068
2,Chicago,IL,Illinois,41.8373,-87.6862
3,Miami,FL,Florida,25.7839,-80.2102
4,Dallas,TX,Texas,32.7936,-96.7662


#### Join Coordinates and Stations <a name="3"></a>

[back to top](#toc)

In [6]:
geo_tv_radio_station_df = tv_radio_station_df.merge(
    geolocation_df,
    how='inner',
    left_on=['communityCity', 'communityState'],
    right_on=['city', 'state_id']
)

geo_tv_radio_station_df = geo_tv_radio_station_df[['city', 'state_name', 'lat', 'lng', 'service', 'status', 'number_of_stations']]
geo_tv_radio_station_df.head(3)

,city,state_name,lat,lng,service,status,number_of_stations
0,St. Louis,Missouri,38.6358,-90.2451,Digital TV,LICENSED,6
1,St. Louis,Missouri,38.6358,-90.2451,Digital Class A TV,LICENSED,1
2,St. Louis,Missouri,38.6358,-90.2451,FM Station,LICENSED,9


#### High Level Aggregation <a name="4"></a>

[back to top](#toc)

In [7]:
high_level_df = geo_tv_radio_station_df.groupby(['state_name']).aggregate('number_of_stations').sum()
high_level_df = pd.DataFrame(high_level_df).reset_index()
high_level_df.head(5)

,state_name,number_of_stations
0,Alabama,409
1,Alaska,201
2,Arizona,288
3,Arkansas,336
4,California,1045


In [8]:
high_level = HighLevelTable(dataframe=high_level_df)
high_level.display()


interactive(children=(IntSlider(value=1, description='min_stations', max=1249, min=1), IntSlider(value=624, de…

#### More Detail <a name="5"></a>

[back to top](#toc)

In [9]:
more_detail = MoreDetailTable(dataframe=geo_tv_radio_station_df)
more_detail.display()

interactive(children=(Dropdown(description='state_name', options=('Alabama', 'Alaska', 'Arizona', 'Arkansas', …

#### Media Station Distribution <a name="6"></a>

[back to top](#toc)


In [10]:
distribution = Distribution(dataframe=geo_tv_radio_station_df)
distribution.display()

interactive(children=(Dropdown(description='service', options=('AM Station', 'Digital Class A TV', 'Digital TV…

#### Geographical Distribution by City <a name="7"></a>

[back to top](#toc)

In [11]:
us_map = USMap(dataframe=geo_tv_radio_station_df)
us_map.display()

interactive(children=(Dropdown(description='service', options=('AM Station', 'Digital Class A TV', 'Digital TV…